# Web-Scrape Section

In [1]:
# Packages
import os #directories
import pandas as pd # dataframe
import re # regular expressions

from bs4 import BeautifulSoup # webscrape
import urllib # webscrape
import requests # webscrape
from urllib.request import urlopen # webscrape
import csv # csv?

filedir = '/Users/Ryan/Desktop/Discord/data'

In [2]:
#Set-up for generalized villager bio
url = "https://animalcrossing.fandom.com/"
pageurl = url + "wiki/Villager_list_(New_Horizons)"
response = requests.get(pageurl)
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
# Look for villager table in Villager list wiki page
all_matches = soup.find_all("table",{"class":"roundy sortable"})

In [4]:
# removes html tags
vinfo = BeautifulSoup(str(all_matches)).get_text()

In [5]:
# Clean text
vinfo2 = vinfo.replace("\n", " ") # replaces /n with space
vinfo2 = re.sub(' +', ' ', vinfo2) # Removes double space
vinfo2 = vinfo2[54:-2] # subtle lazy cleaning
vinfo2 = re.sub('♂', 'Male', vinfo2) # Replaces ♂ with male
vinfo2 = re.sub('♀', 'Female', vinfo2) # Replaces ♂ with male

In [6]:
# Names - returns list
names = re.findall(r'\n\n\n\w+.+\s', vinfo) #pattern to find names

# catchphrases - returns list - looks for string in quotes
catchphrase = re.findall(r'\"(.+?)\"', vinfo2)

# Gender, personality, species, birthday - returns tuple
other = re.findall(r'(Male|Female)\s(Cranky|Jock|Lazy|Smug|Normal|Peppy|Snooty|Sisterly)\s(Alligator|Anteater|Bear|Bird|Bull|Cat|Chicken|Cow|Cub|Deer|Dog|Duck|Eagle|Elephant|Frog|Goat|Gorilla|Hamster|Hippo|Horse|Kangaroo|Koala|Lion|Monkey|Mouse|Octopus|Ostrich|Penguin|Pig|Rabbit|Rhino|Sheep|Squirrel|Tiger|Wolf)\s(January \d{1,2}|February \d{1,2}|March \d{1,2}|April \d{1,2}|May \d{1,2}|June \d{1,2}|July \d{1,2}|August \d{1,2}|September \d{1,2}|October \d{1,2}|November \d{1,2}|December \d{1,2})',vinfo2)

In [7]:
# Looking at first tuple in list other:
other[0]

('Male', 'Cranky', 'Bird', 'January 27')

In [8]:
#Empty list to append to
gender = []
person = []
species = []
bday = []

# Iterate over tuple
for item in other:
    gender.append(item[0])
    person.append(item[1])
    species.append(item[2])
    bday.append(item[3])
    
# Clean and remove the \n from names with list comprehension
fix_names = [s[3:-1] for s in names]

In [9]:
# Create draft dataframe
villager = {'Name': fix_names,
            'Gender': gender,
           'Personality': person,
           'Specie':species,
           'Birthday': bday,
           'Catchphrase': catchphrase
          }
df=pd.DataFrame.from_dict(villager)

In [10]:
df.head(3)

,Name,Gender,Personality,Specie,Birthday,Catchphrase
0,Admiral,Male,Cranky,Bird,January 27,aye aye
1,Agent S,Female,Peppy,Squirrel,July 2,sidekick
2,Agnes,Female,Sisterly,Pig,April 21,snuffle


# Iterate Through Each Villager

In [11]:
url = "https://animalcrossing.fandom.com/wiki/"

In [12]:
# Turn df columns into list

v_name = df.Name.to_list()
v_gender = df.Gender.to_list()
v_personality = df.Personality.to_list()
v_specie = df.Specie.to_list()
v_bday = df.Birthday.to_list()
v_phrase = df.Catchphrase.to_list()

In [13]:
# purpose of v_name2 is for the scraper to iterate through the wiki due to the wiki's naming issues

# Characters have two-part name: Kid Kat and Agent S
v_name2 = [name.replace(" ", "_") for name in v_name]

# Fix character names that have two different names depending on countries
v_name2 = [name.replace("JacobNAJakeyPAL", "Jacob") for name in v_name2]
v_name2 = [name.replace("SporkNACracklePAL", "Spork") for name in v_name2]

# Manual fixes: for villager Anchovy
for n, i in enumerate(v_name2):
    if i == 'Anchovy':
        v_name2[n] = 'Anchovy_(villager)'
        
# Aurora fix
for n, i in enumerate(v_name2):
    if i == 'Aurora':
        v_name2[n] = 'Aurora_(penguin)'
        
# Apple fix
for n, i in enumerate(v_name2):
    if i == 'Apple':
        v_name2[n] = 'Apple_(villager)'
        
# Carmen fix
for n, i in enumerate(v_name2):
    if i == 'Carmen':
        v_name2[n] = 'Carmen_(rabbit)'
        
# Cherry fix
for n, i in enumerate(v_name2):
    if i == 'Cherry':
        v_name2[n] = 'Cherry_(villager)'

# Hazel fix
for n, i in enumerate(v_name2):
    if i == 'Hazel':
        v_name2[n] = 'Hazel_(New_Leaf)'
        
# June Fix        
for n, i in enumerate(v_name2):
    if i == 'June':
        v_name2[n] = 'June_(villager)'
        
# Snooty Fix        
for n, i in enumerate(v_name2):
    if i == 'Snooty':
        v_name2[n] = 'Snooty_(villager)'
        
# Sally Fix        
for n, i in enumerate(v_name2):
    if i == 'Sally':
        v_name2[n] = 'Sally_(Wild_World)'       

In [14]:
# Create villager url list to iterate through
urllist = []

for x in v_name2:
    urllist.append(url + x)

In [15]:
# In-depth Scraping to add-onto draft

# Empty list to append to
skill = []
goal = []
coffee = []
milk = []
sugar = []
style = []
song = []
pngurl = []

# For each url in the url list:
for x in urllist:
    response = requests.get(x) # takes in villager url
    page = response.text # setup
    soup = BeautifulSoup(page, 'html.parser') # grabs doc for each villager
    print(x)
    
    all_matches = soup.find_all("div",{"class":"pi-item pi-data pi-item-spacing pi-border-color"}) # Finds the wiki desc table
    more_info = BeautifulSoup(str(all_matches)).get_text() # Removes HTML tags from doc
    
    # Test Pre-processing
    more_info = more_info.replace("\n", " ") # replaces /n with space
    more_info = re.sub(' +', ' ', more_info) # Removes double space
    more_info = more_info[2:-2] # Lazy edit remove space and bracket from head and tail
    
    # Look for png url link in doc
    all_matches = soup.find_all("figure",{"class":"pi-item pi-image"})
    
    png = re.findall(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', str(all_matches))
    
    #  take the first http png link from the regular expression
    pngurl.append(png[0])
    
    # Appends respective info to list
    '''
    skill.append(''.join(re.findall(r"(?i)Skill (.+?),", more_info))) #Pattern: Phrase after Skill
    goal.append(''.join(re.findall(r"(?i)Goal (.+?),", more_info))) # Pattern: Phrase after Goal
    coffee.append(''.join(re.findall(r"(?i)Coffee (.+?),", more_info))) # Pattern: Phrase after Coffee
    style.append(''.join(re.findall(r"(?i)Style (.+?),", more_info))) # Pattern: Phrase after Style
    song.append(''.join(re.findall(r"(?i)Favorite song (.+?),", more_info))) # Pattern: Phrase after favorite song
    milk.append(''.join(re.findall(r"(?i)(?:[\w\s]+milk|None at all|Unknown)", more_info))) # Pattern: phrase that ends with milk; may return more than 1 in a list so take the first
    sugar.append(''.join(re.findall(r"(?i)(?:[\w\s]+sugar|None at all|Unknown)", more_info))) # pattern: phrase that ends with sugar; may return more than 1 in a list
    '''
    skill.append(re.findall(r"(?i)Skill (.+?),", more_info)[0]) #Pattern: Phrase after Skill
    goal.append(re.findall(r"(?i)Goal (.+?),", more_info)[0]) # Pattern: Phrase after Goal
    coffee.append(re.findall(r"(?i)Coffee (.+?),", more_info)[0]) # Pattern: Phrase after Coffee
    style.append(''.join(re.findall(r"(?i)Style (.+?),", more_info))) # Pattern: Phrase after Style ; some may be unknown so no list index
    song.append(''.join(re.findall(r"(?i)Favorite song (.+?),", more_info))) # Pattern: Phrase after favorite song; some may be unknown so no list index
    milk.append(re.findall(r"(?i)(?:[\w\s]+milk|None at all|Unknown)", more_info)[0]) # Pattern: phrase that ends with milk; may return more than 1 in a list so take the first
    sugar.append(re.findall(r"(?i)(?:[\w\s]+sugar|None at all|Unknown)", more_info)[0]) # pattern: phrase that ends with sugar; may return more than 1 in a list

https://animalcrossing.fandom.com/wiki/Admiral
https://animalcrossing.fandom.com/wiki/Agent_S
https://animalcrossing.fandom.com/wiki/Agnes
https://animalcrossing.fandom.com/wiki/Al
https://animalcrossing.fandom.com/wiki/Alfonso
https://animalcrossing.fandom.com/wiki/Alice
https://animalcrossing.fandom.com/wiki/Alli
https://animalcrossing.fandom.com/wiki/Amelia
https://animalcrossing.fandom.com/wiki/Anabelle
https://animalcrossing.fandom.com/wiki/Anchovy_(villager)
https://animalcrossing.fandom.com/wiki/Ankha
https://animalcrossing.fandom.com/wiki/Angus
https://animalcrossing.fandom.com/wiki/Anicotti
https://animalcrossing.fandom.com/wiki/Annalisa
https://animalcrossing.fandom.com/wiki/Annalise
https://animalcrossing.fandom.com/wiki/Antonio
https://animalcrossing.fandom.com/wiki/Apollo
https://animalcrossing.fandom.com/wiki/Apple_(villager)
https://animalcrossing.fandom.com/wiki/Astrid
https://animalcrossing.fandom.com/wiki/Audie
https://animalcrossing.fandom.com/wiki/Aurora_(penguin)
h

https://animalcrossing.fandom.com/wiki/Jacob
https://animalcrossing.fandom.com/wiki/Jacques
https://animalcrossing.fandom.com/wiki/Jambette
https://animalcrossing.fandom.com/wiki/Jay
https://animalcrossing.fandom.com/wiki/Jeremiah
https://animalcrossing.fandom.com/wiki/Jitters
https://animalcrossing.fandom.com/wiki/Joey
https://animalcrossing.fandom.com/wiki/Judy
https://animalcrossing.fandom.com/wiki/Julia
https://animalcrossing.fandom.com/wiki/Julian
https://animalcrossing.fandom.com/wiki/June_(villager)
https://animalcrossing.fandom.com/wiki/Kabuki
https://animalcrossing.fandom.com/wiki/Katt
https://animalcrossing.fandom.com/wiki/Keaton
https://animalcrossing.fandom.com/wiki/Ken
https://animalcrossing.fandom.com/wiki/Ketchup
https://animalcrossing.fandom.com/wiki/Kevin
https://animalcrossing.fandom.com/wiki/Kid_Cat
https://animalcrossing.fandom.com/wiki/Kidd
https://animalcrossing.fandom.com/wiki/Kiki
https://animalcrossing.fandom.com/wiki/Kitt
https://animalcrossing.fandom.com/wiki

https://animalcrossing.fandom.com/wiki/Tia
https://animalcrossing.fandom.com/wiki/Tiffany
https://animalcrossing.fandom.com/wiki/Timbra
https://animalcrossing.fandom.com/wiki/Tipper
https://animalcrossing.fandom.com/wiki/Tom
https://animalcrossing.fandom.com/wiki/Truffles
https://animalcrossing.fandom.com/wiki/Tucker
https://animalcrossing.fandom.com/wiki/Tutu
https://animalcrossing.fandom.com/wiki/Twiggy
https://animalcrossing.fandom.com/wiki/Tybalt
https://animalcrossing.fandom.com/wiki/Ursala
https://animalcrossing.fandom.com/wiki/Velma
https://animalcrossing.fandom.com/wiki/Vesta
https://animalcrossing.fandom.com/wiki/Vic
https://animalcrossing.fandom.com/wiki/Victoria
https://animalcrossing.fandom.com/wiki/Violet
https://animalcrossing.fandom.com/wiki/Vivian
https://animalcrossing.fandom.com/wiki/Vladimir
https://animalcrossing.fandom.com/wiki/Wade
https://animalcrossing.fandom.com/wiki/Walker
https://animalcrossing.fandom.com/wiki/Walt
https://animalcrossing.fandom.com/wiki/Wart_

In [16]:
# List Cleaning - in milk and sugar list, there are answers like no sugar and none at all. Need to keep it consistent

# If none is in Milk list, replace item with 'No Milk'
for n, item in enumerate(milk):
    if "none" in item.lower():
        milk[n] = 'No Milk'
        
# If none is in Sugar list, replace item with 'No Sugar'
for n, item in enumerate(sugar):
    if "none" in item.lower():
        sugar[n] = 'No Sugar'

In [17]:
# Create dataframe
villager = {'Name': fix_names,
            'Gender': gender,
           'Personality': person,
           'Specie':species,
           'Birthday': bday,
           'Catchphrase': catchphrase,
            'Skill': skill,
            'Goal': goal,
            'Coffee': coffee,
            'Milk': milk,
            'Sugar': sugar,
            'Style': style,
            'Favorite Song': song,
            'pngurl': pngurl
          }
df=pd.DataFrame.from_dict(villager)

# Data Cleaning

In [18]:
# Replace empty fields with unknown
df.replace(r'^\s*$', 'Unknown', regex=True)

,Name,Gender,Personality,Specie,Birthday,Catchphrase,Skill,Goal,Coffee,Milk,Sugar,Style,Favorite Song,pngurl
0,Admiral,Male,Cranky,Bird,January 27,aye aye,Writing about pickles,Fisherman,Blue Mountain,The regular amount of milk,2 spoonfuls of sugar,Unknown,Steep Hill,https://vignette.wikia.nocookie.net/animalcros...
1,Agent S,Female,Peppy,Squirrel,July 2,sidekick,Limboing,Dancer,Mocha,The regular amount of milk,Two spoonfuls of sugar,Sporty,DJ K.K.,https://vignette.wikia.nocookie.net/animalcros...
2,Agnes,Female,Sisterly,Pig,April 21,snuffle,Cramming for tests,Pro Golfer,Kilimanjaro,Lots of milk,Three spoonfuls of sugar,Rock 'n' roll,K.K. House,https://vignette.wikia.nocookie.net/animalcros...
3,Al,Male,Lazy,Gorilla,October 18,Ayyeeee,Jumping rope,Journalist,Mocha,Lots of milk,Three spoonfuls of sugar,Basic,Steep Hill,https://vignette.wikia.nocookie.net/animalcros...
4,Alfonso,Male,Lazy,Alligator,June 9,it'sa me,Fishing,Cartoonist,Mocha,The regular amount of milk,Two spoonfuls of sugar,Basic,Forest Life,https://vignette.wikia.nocookie.net/animalcros...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,Willow,Female,Snooty,Sheep,November 26,bo peep,Playing Old Maid,Designer,Mocha,No milk,No sugar,Cute,K.K. Étude,https://vignette.wikia.nocookie.net/animalcros...
387,Wolfgang,Male,Cranky,Wolf,November 25,snarrrl,Memorizing stuff,Journalist,Blend,Lots of milk,Three spoonfuls of sugar,Modern,K.K. Song,https://vignette.wikia.nocookie.net/animalcros...
388,Yuka,Female,Snooty,Koala,July 20,tsk tsk,Staying up late,Journalist,Kilimanjaro,Lots of milk,Three spoonfuls of sugar,Modern,Soulful K.K.,https://vignette.wikia.nocookie.net/animalcros...
389,Zell,Male,Smug,Deer,June 7,pronk,Calculating tips,Movie director,Blend,Just a little bit of milk,One spoonful of sugar,Modern,K.K. D&B,https://vignette.wikia.nocookie.net/animalcros...


In [19]:
# Create new column search for searching purposes
df['search'] = df.Name.apply(lambda x: (x.replace(" ", "")).lower())

In [20]:
# Remove trailing space from each column
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [21]:
# Write to csv
df.to_csv(filedir + '/villager_data.csv', index = None, header = True)